# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset from the main folder with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo.list_database_names()

['admin',
 'autosaurus',
 'classDB',
 'class_db',
 'config',
 'epa',
 'fruits_DB',
 'gardenDB',
 'local',
 'met',
 'travel_db',
 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [6]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('63d35ad5039f8bcce26df679'),
 'FHRSID': 1349980,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000084064',
 'BusinessName': 'Boodles',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': '3 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DH',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2021-06-14T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.378536', 'latitude': '51.150879'},
 'RightToReply': '',
 'Distance': 4587.638929767669,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode':

In [7]:
# assign the collection to a variable
estabs = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

# Insert the new restaurant into the collection

estabs.insert_one(new_restaurant)

In [9]:
# Check that the new restaurant was inserted
[x for x in estabs.find({'BusinessName':'Penang Flavours'})]

[{'_id': ObjectId('63d35ade568b787433b16dc1'),
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': '',
  'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'PostCode': 'SE18 7DY',
  'Phone': '',
  'LocalAuthorityCode': '511',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
  'SchemeType': 'FHRS',
  'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
  'RightToReply': '',
  'Distance': 4623.972328074718,
  'NewRatingPending': True}]

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [10]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
def find_BusinessTypeID(x):
    BusinessTypeID = estabs.find_one({'BusinessType':f'{x}'}, ['BusinessTypeID','BusinessType'])['BusinessTypeID']
    BusinessType = estabs.find_one({'BusinessType':f'{x}'}, ['BusinessTypeID','BusinessType'])['BusinessType']
    print('''BusinessType: {}
BusinessTypeID: {}'''.format(BusinessType, BusinessTypeID))
    return BusinessTypeID

In [11]:
target_bustype = find_BusinessTypeID('Restaurant/Cafe/Canteen')
target_bustype

BusinessType: Restaurant/Cafe/Canteen
BusinessTypeID: 1


1

3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
estabs.update_one(
    {'BusinessName':'Penang Flavours'},
    {'$set':
        {'BusinessTypeID':target_bustype}
        }
)

In [13]:
# Confirm that the new restaurant was updated
estabs.find_one({'BusinessName':'Penang Flavours'})['BusinessTypeID']

1

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
len([x for x in estabs.find({'LocalAuthorityName':'Dover'})])

994

In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
estabs.delete_many({'LocalAuthorityName':'Dover'})

In [16]:
# Check if any remaining documents include Dover
[x['BusinessName'] for x in estabs.find({'LocalAuthorityName':'Dover'})]

[]

In [17]:
# Check that other documents remain with 'find_one'
estabs.find_one()

{'_id': ObjectId('63d35ad5039f8bcce26df95e'),
 'FHRSID': 289353,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6QH',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2016-09-13T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.194762', 'latitude': '51.085797'},
 'RightToReply': '',
 'Distance': 4591.871474174837,
 'NewRatingPending': False,
 'meta': {'

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude
estabs.update_many({},[
    {'$set':
        {'geocode.longitude':
            {'$toDouble':'$geocode.longitude'}
        }
    }
])

In [19]:
# Change the data type from String to Decimal for latitude
estabs.update_many({},[
    {'$set':
        {'geocode.latitude':
            {'$toDouble':'$geocode.latitude'}
        }
    }
])

In [27]:
# Check that the coordinates are now numbers
estabs.find_one({},{'geocode':1})

{'_id': ObjectId('63d35ad5039f8bcce26df95e'),
 'geocode': {'longitude': 1.194762, 'latitude': 51.085797}}